In [1]:
%matplotlib inline

In [2]:
from bs4 import BeautifulSoup
from collections import defaultdict, OrderedDict

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, roc_auc_score, plot_roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier

from imblearn.over_sampling import RandomOverSampler, ADASYN

import os
import re

## Get metadata for each subject

In [3]:
with open("cm_table.html", "r") as f:
    table = f.read()

soup = BeautifulSoup(table, "html.parser")

rows = soup.find_all(class_="powerTable")[1].tbody.find_all("tr")[3:523]

In [4]:
cols = defaultdict(list)
for row in rows:
    text_list = list(row.stripped_strings)
    if len(text_list) == 13:
        text_list.insert(7, 'na') # insert so list is standard size when that column was empty on the webpage
    cols["study"].append(text_list[2])
    cols["id"].append(text_list[3].replace("_", ""))
    cols["age"].append(float(text_list[8]))
    cols["gender"].append(text_list[10])
    cols["label"].append(text_list[11])

In [5]:
metadata = pd.DataFrame(cols)

## Load the connectivity matrices

In [6]:
def get_conn_matrices():
    file_names = os.listdir("data/ADHD200_CC200")

    cm_file_re = r"^\S+connectivity_matrix_file\.txt$"

    conn_matrices = OrderedDict()
    for file_name in file_names:
        if re.match(cm_file_re, file_name):
            id_ = "".join(file_name.split("_")[:-3])
        
            cm = np.empty((190,190))
            with open("data/ADHD200_CC200/{}".format(file_name)) as f:
                for idx, row in enumerate(f):
                    row = row.strip().split(" ")
                    row = list(map(np.float, row))
                    cm[idx, :] = row
        
            conn_matrices[id_] = cm
    return conn_matrices

In [7]:
def get_regions():
    """
    Gets the names of the regions (in order of appearance in connectivity matrix). 
    All files have the same order of regions, so we only to need to get this once.
    Some region names are repeated because there are multiple points within that region,
        so numbers are appended to the region names to distinguish them.
    
    returns a list of strings
    """
    regions_path = "data/ADHD200_CC200/KKI_1018959_region_names_full_file.txt"
    regions = []
    with open(regions_path, "r") as f:
        regions = [region.strip().replace(" ", "_") for region in f]
    names = defaultdict(int)
    distinct_region_names = []
    for region in regions:
        distinct_region_names.append(region+"_"+str(names[region]))
        names[region] += 1
    return distinct_region_names

In [8]:
region_names = get_regions()

conn_matrices = get_conn_matrices()

## data prep

In [9]:
def flatten_conn_matrices(conn_matrices_dict, region_names):
    """
    Flatten a cm dictionary (mapping subjects to connectivity matrices), such that each unique value in the
        connectivity matrix is a column feature in a row.
    Returns: 1) a numpy array where each row represents a subject with each column a feature;
             2) a list of the subject ids in the order they appear in the feature array;
             3) a list of the feature names in the order they appear in the feature array.
    The subjects list holds the row labels, feature_names list holds column labels.
    """
    subjects = list(conn_matrices_dict.keys())
    num_rows = len(subjects)
    features = np.empty((num_rows, 17955))
    
    # adjacency matrices have duplicate values, only need values from half of the matrix (and don't need diagonal)
    # np.tril_indices() returns indices of unique values
    row_idxs, col_idxs = np.tril_indices(190, k=-1)
    for idx, subject in enumerate(subjects):
        cm = conn_matrices_dict[subject]
        row = np.array([cm[row_idx, col_idx] for row_idx, col_idx in zip(row_idxs, col_idxs)])
        features[idx, :] = row
    
    feature_names = [region_names[row_idx]+"_to_"+region_names[col_idx] 
                     for row_idx, col_idx in zip(row_idxs, col_idxs)]
    
    return features, subjects, feature_names

In [10]:
features, subjects, feature_names = flatten_conn_matrices(conn_matrices, region_names)

In [11]:
# free up some memory
del conn_matrices

In [12]:
def sort_metadata(metadata, subjects):
    """
    Sorts a metadata dataframe so that the order is the same as the order of subjects in the subjects list.
    :arg metadata: dataframe with ADHD200 metadata
    :arg subjects: a list of subjects of specific order
    """
    metadata_ids = metadata["id"].values
    subjects_order_in_metadata = [np.where(metadata_ids==subject)[0][0] for subject in subjects]
    metadata_subject_sort = metadata.iloc[subjects_order_in_metadata, :]
    return metadata_subject_sort

In [13]:
# sort the metadata so that the order is the same as in the feature matrix
metadata_sorted = sort_metadata(metadata, subjects)

In [14]:
# add one hot vectors for each of the ADHD labels
adhd = [0 if label == "Typically Developing" else 1 for label in metadata_sorted["label"]]
metadata_sorted = metadata_sorted.assign(adhd=adhd)
metadata_sorted.drop(columns="label", inplace=True)

In [15]:
metadata_sorted

,study,id,age,gender,adhd
442,ADHD200_CC200,Peking23446674,14.58,Male,1
143,ADHD200_CC200,NYU2054438,8.11,Male,1
47,ADHD200_CC200,KKI2018106,11.66,Male,0
477,ADHD200_CC200,Pittsburgh0016067,17.91,Female,0
395,ADHD200_CC200,Peking21916266,13.17,Male,0
...,...,...,...,...,...
160,ADHD200_CC200,NYU2983819,12.28,Female,1
211,ADHD200_CC200,NYU0010028,9.42,Male,1
250,ADHD200_CC200,NYU0010093,15.21,Female,0
399,ADHD200_CC200,Peking27407032,13.42,Male,0


In [16]:
def most_correlated_features(features, metadata, feature_names, p_val=.01):
    """
    returns a DataFrame with a subset of the features which have a correlation p value less than the specified cutoff
    :arg features: numpy feature matrix, sorted in the same order as the metadata.
    :arg target: DataFrame with target and ids, sorted in the same order as the feature matrix.
    :arg feature_names: the names of the features in the feature matrix, same order.
    :arg p_val: the maximum p value for a feature to be included.
    """
    # get the p values for correlations. lower is better!
    target=metadata["adhd"]
    correlation_p_vals = np.array([pearsonr(features[:,col], target)[1] for col in range(features.shape[1])])
    # get the order of columns which are most correlated with having adhd
    corr_p_vals_argsort = correlation_p_vals.argsort()
    # the number of features with correlation p values less than the cutoff
    num_features = np.count_nonzero(correlation_p_vals < p_val)
    # get the indices of features of features with p vals less than the cutoff
    most_correlated = corr_p_vals_argsort[:num_features]
    
    features_most_correlated = features[:, most_correlated]
    feature_names_most_correlated = [feature_names[idx] for idx in most_correlated]
    
    # make features dataframe with the smaller features
    X = pd.DataFrame(features_most_correlated, columns=feature_names_most_correlated)
    X = X.assign(id=metadata["id"].values)
    X = X.assign(gender=metadata["gender"].values)
    X = X.assign(gender=pd.get_dummies(X["gender"], drop_first=True)["Male"])
    X = X.assign(age=metadata["age"].values)
    X = X.assign(adhd=target.values)
    cols = list(X.columns)
    col_order = [cols[-4]] + [cols[-1]] + [cols[-2]] + [cols[-3]] + cols[:-4]
    X = X[col_order]
    
    return X

In [17]:
X = most_correlated_features(features, metadata_sorted, feature_names, p_val=.001)
print(X.shape[1]-2, "features remaining")

115 features remaining


In [19]:
"""
Seperating by gender doesn't help classification. Interesting side note: there are more males in the data set
    because males are more likely to be diagnosed with ADHD.
    
This option is only compatible with the `all_adhd` flag (above) set to False
"""
seperate_gender = False
if seperate_gender:
    assert not all_adhd, "only set up to process data from the combined adhd flag"
    X_male = X[X["gender"]==1].drop(columns="gender")
    X_female = X[X["gender"]==0].drop(columns="gender")    

## Split data into training and hold out

In [20]:
# try predicting males (more subjects)
if seperate_gender:
    X_train, X_test, y_train, y_test = train_test_split(
            X_male.drop(columns=["adhd"]), X_male["adhd"], test_size=.2, random_state=2)

    X_train_ids = X_train["id"].values
    X_train = X_train.drop(columns=["id"])

    X_test_ids = X_test["id"].values
    X_test = X_test.drop(columns=["id"])
    
else:
    X_train, X_test, y_train, y_test = train_test_split(
            X.drop(columns=["adhd"]), X["adhd"], test_size=.2, random_state=2)

    X_train_ids = X_train["id"].values
    X_train = X_train.drop(columns=["id"])

    X_test_ids = X_test["id"].values
    X_test = X_test.drop(columns=["id"])

In [21]:
# standard scale the data
scale = True
if scale:
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

In [22]:
# percentage of subjects with adhd in the training and testing sets
print("train positive class ratio: {:.3f}".format(np.count_nonzero(y_train == 1)/len(y_train)))
print("test positive class ratio: {:.3f}".format(np.count_nonzero(y_test == 1)/len(y_test)))

train positive class ratio: 0.365
test positive class ratio: 0.365


## Function to print nice confusion matrices

In [24]:
# adapted from the class-imbalance notebook in the Metis curriculum
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=18, percent=True):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
    percent: bool
        whether to output percentages for each true class rather than raw values
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    if percent:
        percent_matrix = np.empty((2,2), dtype=float)
        for row in range(2):
            percent_matrix[row, :] = confusion_matrix[row, :]/np.sum(confusion_matrix[row, :])
        confusion_matrix = percent_matrix
    
    df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names)
    fig = plt.figure(figsize=figsize)
    try:
        if percent:
            heatmap = sns.heatmap(df_cm, annot=True, fmt=".1%")
        else:
            heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

## Modeling

In [25]:
# trains the specified models and prints a confusion matrix for test set predictions
def test_models(models, X_train, y_train, X_test, y_test):
    for model_name in list(models.keys()):
        m = models[model_name]
        m.fit(X_train, y_train)
        
        preds = m.predict(X_test)
        
        roc_auc = roc_auc_score(y_test, preds)
        
        print("{}:".format(model_name))
        print("train acc = {:.3f}".format(m.score(X_train, y_train)))
        print("test acc = {:.3f}\n".format(m.score(X_test, y_test)))
        print("ROC AUC = {:.3f}".format(roc_auc))
        print("-----------------------")

In [26]:
models = {"Logistic regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(n_neighbors=5),
          "SVM": svm.SVC(kernel="rbf"),
          "Naive Bayes": GaussianNB(), 
          "Random forest": RandomForestClassifier(max_depth=4), 
          "Gradient boosting machine": LGBMClassifier()
         }
test_models(models, X_train, y_train, X_test, y_test)

Logistic regression:
train acc = 0.899
test acc = 0.692

ROC AUC = 0.674
-----------------------
KNN:
train acc = 0.774
test acc = 0.692

ROC AUC = 0.629
-----------------------
SVM:
train acc = 0.957
test acc = 0.712

ROC AUC = 0.661
-----------------------
Naive Bayes:
train acc = 0.760
test acc = 0.721

ROC AUC = 0.702
-----------------------
Random forest:
train acc = 0.897
test acc = 0.731

ROC AUC = 0.643
-----------------------
Gradient boosting machine:
train acc = 1.000
test acc = 0.702

ROC AUC = 0.648
-----------------------


## Oversample the minority class
All of these classifiers have horrible positive class precision due to small positive class. If seperating subjects by gender, males already have an almost even split, this won't do anything.

In [27]:
X_train_resampled, y_train_resampled = RandomOverSampler(random_state=0).fit_sample(X_train, y_train)

In [28]:
models = {"Logistic regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(n_neighbors=5),
          "SVM": svm.SVC(kernel="rbf"),
          "Naive Bayes": GaussianNB(), 
          "Random forest": RandomForestClassifier(), 
          "Gradient boosting machine": LGBMClassifier()
         }
test_models(models, X_train_resampled, y_train_resampled, X_test, y_test)

Logistic regression:
train acc = 0.909
test acc = 0.683

ROC AUC = 0.677
-----------------------
KNN:
train acc = 0.818
test acc = 0.654

ROC AUC = 0.621
-----------------------
SVM:
train acc = 0.981
test acc = 0.760

ROC AUC = 0.738
-----------------------
Naive Bayes:
train acc = 0.763
test acc = 0.712

ROC AUC = 0.695
-----------------------
Random forest:
train acc = 1.000
test acc = 0.702

ROC AUC = 0.642
-----------------------
Gradient boosting machine:
train acc = 1.000
test acc = 0.750

ROC AUC = 0.697
-----------------------


#### Synthetic oversampling:

In [29]:
X_adasyn, y_adasyn = ADASYN(random_state=0).fit_sample(X_train, y_train)

In [30]:
models = {"Logistic regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(n_neighbors=5),
          "SVM": svm.SVC(kernel="rbf"),
          "Naive Bayes": GaussianNB(), 
          "Random forest": RandomForestClassifier(), 
          "Gradient boosting machine": LGBMClassifier()
         }
test_models(models, X_adasyn, y_adasyn, X_test, y_test)

Logistic regression:
train acc = 0.906
test acc = 0.712

ROC AUC = 0.700
-----------------------
KNN:
train acc = 0.689
test acc = 0.529

ROC AUC = 0.612
-----------------------
SVM:
train acc = 0.992
test acc = 0.779

ROC AUC = 0.748
-----------------------
Naive Bayes:
train acc = 0.767
test acc = 0.740

ROC AUC = 0.734
-----------------------
Random forest:
train acc = 1.000
test acc = 0.769

ROC AUC = 0.723
-----------------------
Gradient boosting machine:
train acc = 1.000
test acc = 0.769

ROC AUC = 0.729
-----------------------


## Neural net

In [30]:
X_train.shape

(416, 115)

In [33]:
mlp = MLPClassifier(hidden_layer_sizes=(5,), max_iter=2000)
mlp.fit(X_train, y_train)
print("train acc = {:.3f}".format(mlp.score(X_train, y_train)))
print("test acc = {:.3f}".format(mlp.score(X_test, y_test)))
print("ROC AUC = {:.3f}".format(roc_auc_score(mlp.predict(X_test), y_test)))
print(confusion_matrix(y_test, mlp.predict(X_test)))

train acc = 0.978
test acc = 0.596
ROC AUC = 0.588
[[40 27]
 [15 22]]
